## 1. Original dataset, no oversampling

In [1]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import statistics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import preprocessor as pre
import pandas as pd
from collections import Counter
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from scipy import sparse
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gianghoang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gianghoang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
train_data = pd.read_csv("Train.csv", sep=',', encoding="utf-8")
test_data = pd.read_csv("Test.csv", sep=',', encoding="utf-8")

In [3]:
X_train_raw = [x[0] for x in train_data[['text']].values]
Y_train = [x[0] for x in train_data[['sentiment']].values]

X_test_raw = [x[0] for x in test_data[['text']].values]
X_test_ID = [x[0] for x in test_data[['id']].values]

In [4]:
Y_train = np.array(Y_train)

In [5]:
tfidf_vectorizer = TfidfVectorizer()

In [6]:
print(X_train_raw[5]) # has hashtag
print(X_train_raw[34]) # has '!'

 if i didnt have you i'd never see the sun. #mtvstars lady gaga	
 happy birthday to my dude @samshoe1! jason aldean bout to be live come the 5th	


In [7]:
english_stopwords = stopwords.words('english') + ["'d"]
negated_token = 'negation'
negation_stopwords = ['not', 'ain', "should've", "could've", 'shouldve', 'couldve', 'aren',"aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't", 
 'don',
 "don't"]
neutral_stopwords = [word for word in english_stopwords if word not in negation_stopwords]
no_apostrophe_stopwords = [re.sub(r'\'', '', str(word)) for word in negation_stopwords]
contrast_language = ['but', 'yet', 'however', 'despite', 'although', 'though', 'contrast']
contrast_token = 'contrast'

def process_tweet(tweet):
    stemmer = PorterStemmer()
    
    tweet = tweet.lower() # lowercase everything
    
    #removing hyperlinks
    tweet = re.sub(r'(http|https)?:\/\/(\S+)', '', str(tweet))
    
    tweet = re.sub(r'#', '', str(tweet)) 
    #remove hashtags
    
    #removing usernames
    tweet = re.sub(r'/(?<!\w)@[\w+]{1,15}\b/', '', str(tweet))
    
    #removing numbers
    tweet = re.sub(r'[0-9]+', '', str(tweet))
    
    #substitute common abbreviation
    tweet = re.sub(r'wtf', "what the fuck", str(tweet))
    tweet = re.sub(r'wth', "what the hell", str(tweet))
    tweet = re.sub(r'rofl', "rolling on floor laughing", str(tweet))
    tweet = re.sub(r'stfu', "shut the fuck up", str(tweet))
    tweet = re.sub(r'btw', "by the way", str(tweet))
    tweet = re.sub(r'lol', "laugh out loud", str(tweet))
    tweet = re.sub(r'hmu', "hit me up", str(tweet))
    tweet = re.sub(r'imo', "in my opinion", str(tweet))
    tweet = re.sub(r'idk', "I don't know", str(tweet))
    tweet = re.sub(r'idc', "I don't care", str(tweet))
    tweet = re.sub(r'icymi', "in case you missed it", str(tweet))
    tweet = re.sub(r'lmk', "let me know", str(tweet))
    tweet = re.sub(r'nvm', "nevermind", str(tweet))
    tweet = re.sub(r'tbh', "to be honest", str(tweet))
    tweet = re.sub(r'tbf', "to be frank", str(tweet))
    tweet = re.sub(r'lmao', "laughing my ass off", str(tweet))
    tweet = re.sub(r'lmfao', "laughing my fucking ass off", str(tweet))
    tweet = re.sub(r'smh', "shaking my head", str(tweet))
    
    #remove repeating characters
    tweet = re.sub(r'(.)1+', r'1', str(tweet))
    
    #remove non-English characters
    tweet = re.sub("[^a-zA-Z0-9\s]+", " ",str(tweet))
        
#     tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
#                                reduce_len=True)
    
#     tokens = tweet_tokenizer.tokenize(tweet)
    tokens = word_tokenize(tweet)
    processed_tweet = []
    
    for token in tokens:
        if len(token) <= 2:
            continue
        if token in negation_stopwords or token in no_apostrophe_stopwords:
            processed_tweet.append(negated_token)
        elif token in contrast_language:
            processed_tweet.append(contrast_token)
        elif (token not in neutral_stopwords and  # remove stopwords
                token not in string.punctuation):  # remove punctuation
            stemmed_token = stemmer.stem(token)  # stemming word
            processed_tweet.append(stemmed_token)
    return " ".join(processed_tweet)

In [8]:
X_train_processed = [process_tweet(tweet) for tweet in X_train_raw]
X_test_processed = [process_tweet(tweet) for tweet in X_test_raw]

In [9]:
def predict_zero_R(tweet):
    return Counter(Y_train).most_common(1)[0][0]
Zero_R_predict = [predict_zero_R(tweet) for tweet in X_test_processed]

# Zero-R baseline
df = pd.DataFrame(columns = ["id", "sentiment"])
for i in range(len(X_test_ID)):
    new_row = {"id": X_test_ID[i], "sentiment": Zero_R_predict[i]}
    df = df.append(new_row, ignore_index = True)
df.to_csv('prediction.csv', index = False)

In [10]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_processed)

#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_test_tfidf = tfidf_vectorizer.transform(X_test_processed)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, Y_train, test_size = 0.20, shuffle = True, random_state = 1)

param_grid_SVC = {'C': [0.01, 0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'linear']}
 
grid_SVC = GridSearchCV(SVC(), param_grid_SVC, refit = True, verbose = 3)
 
# fitting the model for grid search
grid_SVC.fit(X_train, y_train)
pred_SVC = grid_SVC.predict(X_test)
 
# print classification report
print(classification_report(y_test, pred_SVC))

print(grid_SVC.best_params_)

param_grid_LR = {'C': [0.01, 0.1, 1, 10, 100],
              'penalty': ['l2'],
              'solver': ['newton-cg', 'lbfgs', 'liblinear'],
                'max_iter':[10000]}
 
grid_LR = GridSearchCV(LogisticRegression(), param_grid_LR, refit = True, verbose = 3)
 
# fitting the model for grid search
grid_LR.fit(X_train, y_train)
pred_LR = grid_LR.predict(X_test)
 
# print classification report
print(classification_report(y_test, pred_LR))

print(grid_LR.best_params_)

In [12]:
#Multinomial
MNBmodel = MultinomialNB()

In [13]:
#Support Vector Machine model
SVCmodel = SVC(C = 1.0, gamma = 1.0, kernel = 'linear', cache_size = 1000)

In [14]:
#Logistic Regression model
LRmodel = LogisticRegression(C = 1.0, multi_class='multinomial', solver='newton-cg', max_iter = 10000)

In [17]:
#K-fold MNB
scaler = preprocessing.MaxAbsScaler()
x_scaled = scaler.fit_transform(X_train_tfidf)

skf = KFold(n_splits=10)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]

    MNBmodel.fit(x_train_fold, y_train_fold)
    pred = MNBmodel.predict(x_test_fold)
    lst_accu_stratified.append(MNBmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])
    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')


List of possible accuracy: [0.6290692342961944, 0.609353507565337, 0.6128440366972477, 0.5876146788990826, 0.5912844036697248, 0.615137614678899, 0.6059633027522936, 0.6357798165137615, 0.6385321100917432, 0.6123853211009175]

Maximum Accuracy That can be obtained from this model is: 63.853211009174316 %

Minimum Accuracy: 58.76146788990826 %

Overall Accuracy: 61.37964026265201 %

Overall Neg Pre: 51.276759197622944 %

Overall Neg Rec: 31.29086731462848 %

Overall Neg F1: 38.40200099927461 %

Overall Neu Pre: 64.42926607573393 %

Overall Neu Rec: 79.23350362629749 %

Overall Neg F1: 70.81119288825431 %

Overall Pos Pre: 56.25845200244937 %

Overall pos Rec: 41.41576303705072 %

Overall pos F1: 47.23443111266597 %


In [19]:
skf = KFold(n_splits=10)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]
    #LR
    LRmodel.fit(x_train_fold, y_train_fold)
    pred = LRmodel.predict(x_test_fold)
    lst_accu_stratified.append(LRmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])

    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')

List of possible accuracy: [0.6703347088491518, 0.6570380559376433, 0.6509174311926605, 0.6513761467889908, 0.6431192660550459, 0.653211009174312, 0.6568807339449542, 0.6692660550458716, 0.6646788990825688, 0.6463302752293578]

Maximum Accuracy That can be obtained from this model is: 67.03347088491518 %

Minimum Accuracy: 64.31192660550458 %

Overall Accuracy: 65.63152581300557 %

Overall Neg Pre: 58.27485969194143 %

Overall Neg Rec: 33.64247622896948 %

Overall Neg F1: 42.1713117770657 %

Overall Neu Pre: 67.15128203982823 %

Overall Neu Rec: 83.91195884481618 %

Overall Neg F1: 74.35030482786199 %

Overall Pos Pre: 64.54025945817669 %

Overall pos Rec: 46.04310032206516 %

Overall pos F1: 53.24585212013909 %


In [ ]:
skf = KFold(n_splits=10)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]

    #SVM
    SVCmodel.fit(x_train_fold, y_train_fold)
    pred = SVCmodel.predict(x_test_fold)
    lst_accu_stratified.append(SVCmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])
    
    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')

In [21]:
#SKF MNB
scaler = preprocessing.MaxAbsScaler()
x_scaled = scaler.fit_transform(X_train_tfidf)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]

    MNBmodel.fit(x_train_fold, y_train_fold)
    pred = MNBmodel.predict(x_test_fold)
    lst_accu_stratified.append(MNBmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])

    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')

List of possible accuracy: [0.6102705181109582, 0.6198991288399817, 0.6211009174311927, 0.6064220183486239, 0.6261467889908257, 0.6137614678899083, 0.634862385321101, 0.6192660550458715, 0.6091743119266055, 0.5972477064220183]

Maximum Accuracy That can be obtained from this model is: 63.48623853211009 %

Minimum Accuracy: 59.72477064220183 %

Overall Accuracy: 61.58151298327087 %

Overall Neg Pre: 51.13731563558201 %

Overall Neg Rec: 30.95448221893749 %

Overall Neg F1: 38.524873277949304 %

Overall Neu Pre: 64.54209496280555 %

Overall Neu Rec: 79.12926711999451 %

Overall Neg F1: 71.0898061820406 %

Overall Pos Pre: 56.107630133287834 %

Overall pos Rec: 41.617771978824756 %

Overall pos F1: 47.75782011017493 %


In [22]:
#SKF Logistic Regression
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]
    #LR
    LRmodel.fit(x_train_fold, y_train_fold)
    pred = LRmodel.predict(x_test_fold)
    lst_accu_stratified.append(LRmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])

    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')

List of possible accuracy: [0.6616231086657497, 0.6629986244841816, 0.6614678899082569, 0.6353211009174312, 0.6688073394495413, 0.6688073394495413, 0.6697247706422018, 0.6490825688073395, 0.6770642201834862, 0.6490825688073395]

Maximum Accuracy That can be obtained from this model is: 67.70642201834862 %

Minimum Accuracy: 63.53211009174312 %

Overall Accuracy: 66.03979531315069 %

Overall Neg Pre: 58.74386489611221 %

Overall Neg Rec: 33.728733733298554 %

Overall Neg F1: 42.829001218024324 %

Overall Neu Pre: 67.36288595808695 %

Overall Neu Rec: 83.95593478573078 %

Overall Neg F1: 74.74685501982763 %

Overall Pos Pre: 64.73404754976791 %

Overall pos Rec: 46.37078414983045 %

Overall pos F1: 54.01813431143625 %


In [24]:
#SKF SVM
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]

    #SVM
    SVCmodel.fit(x_train_fold, y_train_fold)
    pred = SVCmodel.predict(x_test_fold)
    lst_accu_stratified.append(SVCmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])

    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')

List of possible accuracy: [0.6414488766620816, 0.6529115084823476, 0.6449541284403669, 0.6233944954128441, 0.6458715596330276, 0.6458715596330276, 0.6486238532110091, 0.6311926605504588, 0.6591743119266055, 0.6307339449541285]

Maximum Accuracy That can be obtained from this model is: 65.91743119266054 %

Minimum Accuracy: 62.33944954128441 %

Overall Accuracy: 64.24176898905897 %

Overall Neg Pre: 51.35766473060147 %

Overall Neg Rec: 40.94238182187057 %

Overall Neg F1: 45.54935442364321 %

Overall Neu Pre: 68.0982563026029 %

Overall Neu Rec: 77.09915141524456 %

Overall Neg F1: 72.3147080108876 %

Overall Pos Pre: 60.51166593135381 %

Overall pos Rec: 50.20247633415561 %

Overall pos F1: 54.849073586321474 %


In [25]:
pred = SVCmodel.predict(X_test_tfidf)
df_SVC_clean = pd.DataFrame(columns = ["id", "sentiment"])
for i in range(len(X_test_ID)):
    new_row = {"id": X_test_ID[i], "sentiment": pred[i]}
    df_SVC_clean = df_SVC_clean.append(new_row, ignore_index = True)
df_SVC_clean.to_csv('prediction_SVC_clean.csv', index = False)

## 2. With oversampling

In [26]:
train_data = pd.read_csv("Train.csv", sep=',', encoding="utf-8")
test_data = pd.read_csv("Test.csv", sep=',', encoding="utf-8")

In [27]:
neutral = train_data[train_data['sentiment'] == 'neutral']
positive = train_data[train_data['sentiment'] == 'positive']
negative = train_data[train_data['sentiment'] == 'negative']

In [28]:
new_pos = positive.append(positive)
new_neg = negative.append(negative.append(negative))
train_data = neutral.append(new_pos.append(new_neg))
train_data = train_data.sample(frac = 1)

In [29]:
X_train_raw = [x[0] for x in train_data[['text']].values]
Y_train = [x[0] for x in train_data[['sentiment']].values]

X_test_raw = [x[0] for x in test_data[['text']].values]
X_test_ID = [x[0] for x in test_data[['id']].values]

In [30]:
Y_train = np.array(Y_train)

In [31]:
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
english_stopwords = stopwords.words('english') + ["'d"]
negated_token = 'negation'
negation_stopwords = ['not', 'ain', "should've", "could've", 'shouldve', 'couldve', 'aren',"aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't", 
 'don',
 "don't"]
neutral_stopwords = [word for word in english_stopwords if word not in negation_stopwords]
no_apostrophe_stopwords = [re.sub(r'\'', '', str(word)) for word in negation_stopwords]
contrast_language = ['but', 'yet', 'however', 'despite', 'although', 'though', 'contrast']
contrast_token = 'contrast'

def process_tweet(tweet):
    stemmer = PorterStemmer()
    
    tweet = tweet.lower() # lowercase everything
    
    #removing hyperlinks
    tweet = re.sub(r'(http|https)?:\/\/(\S+)', '', str(tweet))
    
    tweet = re.sub(r'#', '', str(tweet)) 
    #remove hashtags
    
    #removing usernames
    tweet = re.sub(r'/(?<!\w)@[\w+]{1,15}\b/', '', str(tweet))
    
    #removing numbers
    tweet = re.sub(r'[0-9]+', '', str(tweet))
    
    #substitute common abbreviation
    tweet = re.sub(r'wtf', "what the fuck", str(tweet))
    tweet = re.sub(r'wth', "what the hell", str(tweet))
    tweet = re.sub(r'rofl', "rolling on floor laughing", str(tweet))
    tweet = re.sub(r'stfu', "shut the fuck up", str(tweet))
    tweet = re.sub(r'btw', "by the way", str(tweet))
    tweet = re.sub(r'lol', "laugh out loud", str(tweet))
    tweet = re.sub(r'hmu', "hit me up", str(tweet))
    tweet = re.sub(r'imo', "in my opinion", str(tweet))
    tweet = re.sub(r'idk', "I don't know", str(tweet))
    tweet = re.sub(r'idc', "I don't care", str(tweet))
    tweet = re.sub(r'icymi', "in case you missed it", str(tweet))
    tweet = re.sub(r'lmk', "let me know", str(tweet))
    tweet = re.sub(r'nvm', "nevermind", str(tweet))
    tweet = re.sub(r'tbh', "to be honest", str(tweet))
    tweet = re.sub(r'tbf', "to be frank", str(tweet))
    tweet = re.sub(r'lmao', "laughing my ass off", str(tweet))
    tweet = re.sub(r'lmfao', "laughing my fucking ass off", str(tweet))
    tweet = re.sub(r'smh', "shaking my head", str(tweet))
    
    #remove repeating characters
    tweet = re.sub(r'(.)1+', r'1', str(tweet))
    
    #remove non-English characters
    tweet = re.sub("[^a-zA-Z0-9\s]+", " ",str(tweet))
        
#     tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
#                                reduce_len=True)
    
#     tokens = tweet_tokenizer.tokenize(tweet)
    tokens = word_tokenize(tweet)
    processed_tweet = []
    
    for token in tokens:
        if len(token) <= 2:
            continue
        if token in negation_stopwords or token in no_apostrophe_stopwords or token in contrast_language:
            continue
        elif token in ['like', 'tomorrow']:
            continue
        elif (token not in neutral_stopwords and  # remove stopwords
                token not in string.punctuation):  # remove punctuation
            stemmed_token = stemmer.stem(token)  # stemming word
            processed_tweet.append(stemmed_token)
    return " ".join(processed_tweet)

In [32]:
X_train_processed = [process_tweet(tweet) for tweet in X_train_raw]
X_test_processed = [process_tweet(tweet) for tweet in X_test_raw]

In [33]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_processed)

#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_test_tfidf = tfidf_vectorizer.transform(X_test_processed)

In [34]:
#K-fold MNB
scaler = preprocessing.MaxAbsScaler()
x_scaled = scaler.fit_transform(X_train_tfidf)

skf = KFold(n_splits=10)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]

    MNBmodel.fit(x_train_fold, y_train_fold)
    pred = BNBmodel.predict(x_test_fold)
    lst_accu_stratified.append(BNBmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])

    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')


NameError: name 'BNBmodel' is not defined

In [ ]:
#K-fold Logistic Regression
skf = KFold(n_splits=10)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]
    #LR
    LRmodel.fit(x_train_fold, y_train_fold)
    pred = LRmodel.predict(x_test_fold)
    lst_accu_stratified.append(LRmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])

    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')

In [ ]:
#K-fold SVM
skf = KFold(n_splits=10)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]

    #SVM
    SVCmodel.fit(x_train_fold, y_train_fold)
    pred = SVCmodel.predict(x_test_fold)
    lst_accu_stratified.append(SVCmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])

    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')

In [ ]:
#SKF MNB
scaler = preprocessing.MaxAbsScaler()
x_scaled = scaler.fit_transform(X_train_tfidf)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]

    MNBmodel.fit(x_train_fold, y_train_fold)
    pred = BNBmodel.predict(x_test_fold)
    lst_accu_stratified.append(BNBmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])

    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')

In [ ]:
#SKF Logistic Regression
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]
    #LR
    LRmodel.fit(x_train_fold, y_train_fold)
    pred = LRmodel.predict(x_test_fold)
    lst_accu_stratified.append(LRmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])

    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')

In [ ]:
#SKF SVM
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
lst_accu_stratified = []
neg_pre = []
neg_re = []
neg_F1 = []
neu_pre = []
neu_re = []
neu_F1 = []
pos_pre = []
pos_re = []
pos_F1 = []
  
for train_index, test_index in skf.split(X_train_tfidf, Y_train):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = Y_train[train_index], Y_train[test_index]

    #SVM
    SVCmodel.fit(x_train_fold, y_train_fold)
    pred = SVCmodel.predict(x_test_fold)
    lst_accu_stratified.append(SVCmodel.score(x_test_fold, y_test_fold))

    matrix = classification_report(y_test_fold, pred, output_dict = True)
    neg_pre.append(matrix['negative']['precision'])
    neg_re.append(matrix['negative']['recall'])
    neg_F1.append(matrix['negative']['f1-score'])
    neu_pre.append(matrix['neutral']['precision'])
    neu_re.append(matrix['neutral']['recall'])
    neu_F1.append(matrix['neutral']['f1-score'])
    pos_pre.append(matrix['positive']['precision'])
    pos_re.append(matrix['positive']['recall'])
    pos_F1.append(matrix['positive']['f1-score'])

    
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      np.mean(lst_accu_stratified)*100, '%')
print('\nOverall Neg Pre:',
      np.mean(neg_pre)*100, '%')
print('\nOverall Neg Rec:',
      np.mean(neg_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neg_F1)*100, '%')
print('\nOverall Neu Pre:',
      np.mean(neu_pre)*100, '%')
print('\nOverall Neu Rec:',
      np.mean(neu_re)*100, '%')
print('\nOverall Neg F1:',
      np.mean(neu_F1)*100, '%')
print('\nOverall Pos Pre:',
      np.mean(pos_pre)*100, '%')
print('\nOverall pos Rec:',
      np.mean(pos_re)*100, '%')
print('\nOverall pos F1:',
      np.mean(pos_F1)*100, '%')